## **Topic Clustering Model Training using subset of the given dataset**

In [ ]:
!pip install transformers
!pip install torch==2.2.0
!pip install tqdm
!pip install scikit-learn

In [ ]:
#!pip install tqdm

In [ ]:
import pandas as pd

df_excel = pd.read_excel('/content/new_file_Description.xlsx')

In [ ]:
import pandas as pd

# Example DataFrame
data = df_excel
df = pd.DataFrame(data)

# Find unique values in the "Description" column
unique_values = df['stage_0.1'].unique()

# Print the unique values
print("Unique Values in Description column:", unique_values)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Example DataFrame
data = df_excel
df = pd.DataFrame(data)

# Use LabelEncoder to encode ministry names
#label_encoder = LabelEncoder()
#df['numeric_label'] = label_encoder.fit_transform(df['Description'])

# Create dynamic entailment mapping based on unique ministries
unique_ministries = df['stage_0.2'].unique()
entailment_mapping = {}

for text, ministry in zip(df['text_column'], df['stage_0.2']):
    # Assign entailment values based on unique ministry
    text = text.replace("\r\n", ' ')
    entailment_mapping[text] = {'ministry': ministry, 'entailment': 1}

# Apply the mapping to create numerical labels based on entailment
df['numeric_label'] = df['text_column'].map(lambda x: entailment_mapping[x]['entailment'])

# Display the resulting DataFrame and entailment_mapping
#print("DataFrame:")
#print(df)

#print("\nEntailment Mapping:")
#print(entailment_mapping)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BartTokenizer, BartForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = str(self.dataframe.iloc[idx]['text_column'])
        label = int(self.dataframe.iloc[idx]['numeric_label'])

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli') #facebook/bart-large-mnli
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli') #facebook/bart-large-mnli

# Create custom datasets and dataloaders
train_dataset = CustomDataset(train_df, tokenizer)
val_dataset = CustomDataset(val_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Training parameters
num_epochs = 50
learning_rate = 5e-5

# Initialize the optimizer and scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
#optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Loss function
loss_fn = CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

    # Validation loop
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Average validation loss: {avg_val_loss}")


In [ ]:
# Example of calculating accuracy
correct_predictions = (torch.argmax(outputs.logits, dim=1) == labels).sum().item()
accuracy = correct_predictions / labels.size(0)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


In [ ]:
import pickle
model_data = {'model': model, 'tokenizer': tokenizer}

with open('fine_tuned_model.pkl', 'wb') as file:
    pickle.dump(model_data, file)

In [ ]:
import pickle
model_data = {'model': model, 'tokenizer': tokenizer}

pickle_file_path = '/content/drive/MyDrive/fine_tuned_model_worksfine.pkl'

with open(pickle_file_path, 'wb') as file:
    pickle.dump(model_data, file)

In [ ]:
# Save the fine-tuned model and tokenizer with torch.save
#torch.save(model.state_dict(), 'fine_tuned_model.pth')
#tokenizer.save_pretrained('./fine_tuned_model')

# Load the fine-tuned model and tokenizer
#loaded_model = TFBartForSequenceClassification.from_pretrained('fine_tuned_model', state_dict=torch.load('fine_tuned_model.pth'))
#loaded_tokenizer = BartTokenizer.from_pretrained('./fine_tuned_model')


## **Topic Clustering system using the created model**

In [ ]:
import pickle
import torch
from transformers import pipeline, TFBartForSequenceClassification, BartTokenizer

# Load the fine-tuned model and tokenizer
#loaded_model = TFBartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
#loaded_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')

# Load the fine-tuned model and tokenizer from the pickle file
with open('/content/drive/MyDrive/fine_tuned_model_worksfine.pkl', 'rb') as file:
  saved_model_data = pickle.load(file)

# Map the model to a CPU device
loaded_model = saved_model_data['model']
loaded_tokenizer = saved_model_data['tokenizer']

loaded_model.to('cpu')

classifier = pipeline("zero-shot-classification", model=loaded_model, tokenizer=loaded_tokenizer)

In [ ]:
def classification (text, stagelabels):
    top5 = []
    candidate_labels = stagelabels #catdf['stage2'] #['bank', 'received', 'right', 'wrong']
    # Tokenize the input string
    tokenizer = loaded_tokenizer

    tokenized_input = tokenizer(text, return_tensors="pt")
    # Now `tokenized_input` contains the necessary tokens for the model
    # You can pass it through the loaded model

    output = classifier(text, candidate_labels, multi_label=True)

    if (len(output['labels'])) > 5:
       for m in range (0,4):
          if output['labels']:
             top5.append(output['labels'][m])
    elif (len(output['labels'])) <= 5 and (len(output['labels'])) != 0:
         top5 = output['labels']
    else:
         top5 = []
    return (output, output['labels'][0], top5)


In [ ]:
def classify_items(json_data, org_code, sequence_to_classify): # instead of org_code stage 3 description can be given
    # Convert JSON data to a list of dictionaries
    data = json.loads(json_data)

    if org_code.startswith("GOV"):
       data = data
       original_data = data.copy()
    else:
       # Filter data by OrgCode
       data = [item for item in data if item['OrgCode'] == org_code]
       original_data = data.copy()
       # data = [item for item in data if item['Parent'] == (item ['Code'] if item['Description'] == stage3_description)] i.e. find the code of the stage3 item description

    #print(data)

    # Initialize result dictionary
    result = {}

    text = sequence_to_classify

    for stage in range(1, 9): # Alter the stage no according to the stage 3 description
        # Get items for the current stage
        stage_items = [item for item in data if item['Stage'] == stage]
        #print(stage_items)

        # If there are no items for the current stage, break the loop
        if not stage_items:
            break

        # Get the descriptions of the items
        descriptions = [item['Description'] for item in stage_items]

        # Classify the descriptions
        output, best_item, top_5_items = classification(text, descriptions)

        # Add the best item to the result
        result[f'Stage {stage}'] = {
            'Best Item': best_item,
            'Top 5 Items': top_5_items
        }
        print(best_item)
        with open('result.json', 'w') as f:
             json.dump(result, f)

        for item in data:
            if item['Description'] == best_item:
               best_item_code = item['Code']
               break
        print(best_item_code)
        # Update data to only include items whose parent is the code of the best item
        data = [item for item in original_data if item['Parent'] == best_item_code]
        print(data)
        print(output)

    return result

In [ ]:
import json

with open('/content/drive/MyDrive/problem_statement_1_and_2/no_pii_grievance.json', 'r') as f:
     grievance_df = json.load(f)

with open('/content/drive/MyDrive/problem_statement_1_and_2/CategoryCode_Mapping_v2.json', 'r') as c:
     # Load JSON data from file
     json_dat = json.load(c)    # Your JSON data

json_data = json.dumps(json_dat)
result = []

gdf = {"code":[], "text":[]}

gr_id =  "CBODT/E/2023/0000001" #"MEAPD/E/2023/0000002" #"CBODT/E/2023/0000001" #"MOLBR/E/2023/0000004" #"MORLY/E/2023/0000001" #"MOLBR/E/2023/0000003"
for id in grievance_df:
    if id['_id'] == gr_id:
       gdf["code"] = id['org_code']
       gdf["text"] = id['subject_content_text']
       break

#id = 0 #Id is the only needed to be given usually the unique id of the grievance.the id will fetch all the details from the DB including org code

sequence_to_classify = gdf['text'] #Text to classify
org_code = gdf['code'] #OrgCode

print(sequence_to_classify)

result = classify_items(json_data, org_code, sequence_to_classify)

print(result)

Central Board of Direct Taxes (Income Tax) >> Direct Taxes >> Search, Seizure Matters,     Reward to informer

PAN No. : X-X-X-X-X
Regional office / Office : Region Andhra Pradesh & Telangana
-----------------------
HAPPY NEW YEAR 2023
SIR I REQUEST THE HONOURABLE MINISTER FOR FINANCE, GOVERNMENT OF INDIA,NEW DELHI TO SETTLE THE CASE ATTACHED WITH CBODT CENTRAL BOARD OF DIRECT TAXES AND BLESS ME THIS YEAR WITH EMPLOYEMENT
Central Board of Direct Taxes (Income Tax)
4414
[{'Code': 4415, 'Description': 'Pension Related', 'OrgCode': 'CBODT', 'Parent': 4414, 'Stage': 2, 'MonitoringCode': None}, {'Code': 4416, 'Description': 'Employee Related/ Services related', 'OrgCode': 'CBODT', 'Parent': 4414, 'Stage': 2, 'MonitoringCode': None}, {'Code': 4417, 'Description': 'Administration related', 'OrgCode': 'CBODT', 'Parent': 4414, 'Stage': 2, 'MonitoringCode': None}, {'Code': 4418, 'Description': 'Corruption/Malpractices related (VCs, employees)', 'OrgCode': 'CBODT', 'Parent': 4414, 'Stage': 2, 'Mo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(sequence_to_classify)
print(org_code)

## **Summarizer model fine tuning for report generation based on dataset**

In [ ]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import torch.optim as optim

# Sample DataFrame (replace this with your actual data)
data = pd.read_excel('/content/cleaned_file_6.xlsx')
df = pd.DataFrame(data)

class CustomSummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_source_length=1024, max_target_length=150):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        source_text = str(self.dataframe.iloc[idx]['text_column'])
        target_text = str(self.dataframe.iloc[idx]['text_column'])

        encoding = self.tokenizer(
            source_text,
            max_length=self.max_source_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        labels = self.tokenizer(
            target_text,
            max_length=self.max_target_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )['input_ids']

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Create custom dataset and dataloaders
train_dataset = CustomSummarizationDataset(df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Training parameters
num_epochs = 3
learning_rate = 1e-5

# Initialize the optimizer and scheduler
optimizer = optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Use SGD optimizer #AdamW(model.parameters(), lr=learning_rate) #optimizer = optim.SGD(model.parameters(), lr=learning_rate)  # Use SGD optimizer

total_steps = len(train_dataloader) * num_epochs

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss}")

# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer')
tokenizer.save_pretrained('/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer')


## **Summarizer system for Report Generation**

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer')
tokenizer = BartTokenizer.from_pretrained('/content/drive/MyDrive/bart_large_cnn_fine_tuned_summarizer')

# Example text for summarization
input_text = "CITIZEN SAYS THAT THERE IS NO PROPER ROAD FACILITY IN THEIR VILLAGE DUE TO THIS THEY ARE UNABLE TO TRAVEL OR WALK ON THIS MUD ROAD AND REQUESTING TO CONSTRUCT NEW ROAD. KINDLY DO THE NEEDFUL."
# Tokenize and generate summary
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
summary_ids = model.generate(**inputs)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:", summary)

## **Additional Independent Translator for Indian Langauges (Except Odia)**

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_hi,
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
#translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

print(translation)

['The head of the United Nations says there is no military solution in Syria']
